In [1]:
from pyspark_llap.sql.session import HiveWarehouseSession
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("LLAP_Test").enableHiveSupport().config("spark.sql.hive.llap", "true").getOrCreate()
spark = SparkSession.builder.appName("Hive_ACID_warehouseconnector").enableHiveSupport().getOrCreate()
hive = HiveWarehouseSession.session(spark).build()

In [2]:
from pyspark.sql.functions import udf, col, lit, array
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType

import time
import pandas as pd
import numpy as np
import datetime
import math
import scipy
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import subprocess
import sys
import pyspark.sql as SQL
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import avg
from pyspark import SparkConf
from pyspark.sql import HiveContext
from pyspark import SparkContext
from pyspark.sql.functions import col,row_number
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import Row
from pyspark.sql.functions import to_timestamp,unix_timestamp,from_unixtime
from pyspark.sql.types import DoubleType, FloatType, BooleanType, LongType, IntegerType, TimestampType, StringType, StructType, StructField, ArrayType

sqlContext.setConf("spark.sql.shuffle.partitions", "2500")
# pd.set_option('display.max_colwidth', -1)

In [3]:
hiveQuery = "SELECT * from mfard_data.CCBDDF150 where cvdcus_dcd_sig_n_x_3 IN ('BrkTot_Tq_Actl', 'AirCondFluidHi_P_Actl',\
'Ignition_Status','Outside_Air_Temp_Stat', 'VehLong2_A_Actl', 'BattULo_U_Actl', 'CoolantFanStepAct', 'ApedPos_Pc_ActlArb', \
'HvacBlwrFront_D_Stat', 'OdometerMasterValue', 'HvacEvap_Te_Rq', 'HvacAirCond_B_Rq','EngAout_N_Actl','AirAmb_Te_ActlFilt',\
'Veh_V_ActlEng', 'GearLvrPos_D_Actl', 'HvacEvap_Te_Actl')"

In [4]:
BDD_F150_2019 = spark.sql(hiveQuery)
BDD_F150_2019.show(20, False)

+-----------------+-------------------------+--------------------+--------------------------+
|cvdcus_vin_d_3   |cvdcus_timestamp_s_3     |cvdcus_dcd_sig_n_x_3|cvdcus_dcd_sig_val_str_x_3|
+-----------------+-------------------------+--------------------+--------------------------+
|1FTEW1C53KFB01079|2019-07-17 22:34:41.0693 |ApedPos_Pc_ActlArb  |0.0                       |
|1FTEW1C53KFB01079|2019-07-17 22:34:41.0693 |EngAout_N_Actl      |0.0                       |
|1FTEW1C53KFB01079|2019-07-17 22:34:41.07243|Veh_V_ActlEng       |0.00                      |
|1FTEW1C53KFB01079|2019-07-17 22:34:41.07264|EngAout_N_Actl      |0.0                       |
|1FTEW1C53KFB01079|2019-07-17 22:34:41.07264|ApedPos_Pc_ActlArb  |0.0                       |
|1FTEW1C53KFB01079|2019-07-17 22:34:41.07391|BattULo_U_Actl      |12.6250                   |
|1FTEW1C53KFB01079|2019-07-17 22:34:41.07537|EngAout_N_Actl      |0.0                       |
|1FTEW1C53KFB01079|2019-07-17 22:34:41.07537|ApedPos_Pc_Actl

In [5]:
BDD_F150_2019.dtypes

[('cvdcus_vin_d_3', 'string'),
 ('cvdcus_timestamp_s_3', 'timestamp'),
 ('cvdcus_dcd_sig_n_x_3', 'string'),
 ('cvdcus_dcd_sig_val_str_x_3', 'string')]

In [6]:
# list of the unique values of column = cvdcus_dcd_sig_n_x_3
# BDD_F150_2019.select('cvdcus_dcd_sig_n_x_3').distinct().rdd.map(lambda r: r[0]).collect()

# another way
BDD_F150_2019.select('cvdcus_dcd_sig_n_x_3').distinct().collect()

[Row(cvdcus_dcd_sig_n_x_3='BrkTot_Tq_Actl'),
 Row(cvdcus_dcd_sig_n_x_3='AirCondFluidHi_P_Actl'),
 Row(cvdcus_dcd_sig_n_x_3='Ignition_Status'),
 Row(cvdcus_dcd_sig_n_x_3='Outside_Air_Temp_Stat'),
 Row(cvdcus_dcd_sig_n_x_3='VehLong2_A_Actl'),
 Row(cvdcus_dcd_sig_n_x_3='BattULo_U_Actl'),
 Row(cvdcus_dcd_sig_n_x_3='CoolantFanStepAct'),
 Row(cvdcus_dcd_sig_n_x_3='ApedPos_Pc_ActlArb'),
 Row(cvdcus_dcd_sig_n_x_3='HvacBlwrFront_D_Stat'),
 Row(cvdcus_dcd_sig_n_x_3='OdometerMasterValue'),
 Row(cvdcus_dcd_sig_n_x_3='HvacEvap_Te_Rq'),
 Row(cvdcus_dcd_sig_n_x_3='HvacAirCond_B_Rq'),
 Row(cvdcus_dcd_sig_n_x_3='EngAout_N_Actl'),
 Row(cvdcus_dcd_sig_n_x_3='AirAmb_Te_ActlFilt'),
 Row(cvdcus_dcd_sig_n_x_3='Veh_V_ActlEng'),
 Row(cvdcus_dcd_sig_n_x_3='GearLvrPos_D_Actl'),
 Row(cvdcus_dcd_sig_n_x_3='HvacEvap_Te_Actl')]

In [7]:
# BDD for MY20
# READ BDD SIGNALS
signal = BDD_F150_2019\
.withColumn('epoch', unix_timestamp("cvdcus_timestamp_s_3").cast("long"))\
.orderBy(['epoch'])\
.groupBy('cvdcus_vin_d_3','epoch','cvdcus_dcd_sig_n_x_3')\
.agg(F.avg('cvdcus_dcd_sig_val_str_x_3').alias('avg'),F.max('cvdcus_dcd_sig_val_str_x_3').alias('max'))\
.orderBy(['cvdcus_vin_d_3','epoch'])\
.select('cvdcus_vin_d_3','epoch',F.col('cvdcus_dcd_sig_n_x_3').alias('signal_name'),F.coalesce('avg','max').alias('signal_value'))\
.drop('avg','max')\

signal.show(10, False)

+-----------------+----------+-------------------+-----------------+
|cvdcus_vin_d_3   |epoch     |signal_name        |signal_value     |
+-----------------+----------+-------------------+-----------------+
|1FTEW1C42KKC86661|1560597070|OdometerMasterValue|7640.0           |
|1FTEW1C42KKC86661|1560597070|GearLvrPos_D_Actl  |Park             |
|1FTEW1C42KKC86661|1560597070|HvacEvap_Te_Rq     |6.5              |
|1FTEW1C42KKC86661|1560597070|BattULo_U_Actl     |13.5625          |
|1FTEW1C42KKC86661|1560597070|EngAout_N_Actl     |617.0909090909091|
|1FTEW1C42KKC86661|1560597070|HvacAirCond_B_Rq   |Yes              |
|1FTEW1C42KKC86661|1560597070|HvacEvap_Te_Actl   |5.875            |
|1FTEW1C42KKC86661|1560597070|AirAmb_Te_ActlFilt |21.5             |
|1FTEW1C42KKC86661|1560597070|ApedPos_Pc_ActlArb |0.0              |
|1FTEW1C42KKC86661|1560597070|BrkTot_Tq_Actl     |0.0              |
+-----------------+----------+-------------------+-----------------+
only showing top 10 rows



In [8]:
signal2=signal\
.groupBy('cvdcus_vin_d_3','epoch')\
.pivot('signal_name')\
.agg(F.max('signal_value'))\
.orderBy(['cvdcus_vin_d_3','epoch'])\

In [9]:
signal2.show(10, False)

+-----------------+----------+------------------+---------------------+------------------+--------------+--------------+-----------------+-----------------+-----------------+----------------+--------------------+----------------+--------------+---------------+-------------------+---------------------+--------------------+-------------+
|cvdcus_vin_d_3   |epoch     |AirAmb_Te_ActlFilt|AirCondFluidHi_P_Actl|ApedPos_Pc_ActlArb|BattULo_U_Actl|BrkTot_Tq_Actl|CoolantFanStepAct|EngAout_N_Actl   |GearLvrPos_D_Actl|HvacAirCond_B_Rq|HvacBlwrFront_D_Stat|HvacEvap_Te_Actl|HvacEvap_Te_Rq|Ignition_Status|OdometerMasterValue|Outside_Air_Temp_Stat|VehLong2_A_Actl     |Veh_V_ActlEng|
+-----------------+----------+------------------+---------------------+------------------+--------------+--------------+-----------------+-----------------+-----------------+----------------+--------------------+----------------+--------------+---------------+-------------------+---------------------+--------------------+-

In [23]:
signal2.count()

7628887

In [25]:
mydf=signal2.toPandas()

In [27]:
mydf.to_csv(r'/s/mfard/myfile.csv', index = False)

# to_parquet('/s/mfard/myfile.parquet')